In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
from os import listdir
from os.path import isfile, join





/Users/eshan/anaconda3/envs/newenvt/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 1600 # total no. of training examples
nb_validation_samples = 400 # total np. of validation examples
epochs = 50
batch_size = 16

In [3]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', # or categorical_crossentropy
              optimizer='rmsprop',# or adagrad
              metrics=['accuracy'])




In [5]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)



In [6]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

print(train_generator.class_indices)

Found 1600 images belonging to 2 classes.
{'dogs': 1, 'cats': 0}


In [8]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 200 images belonging to 2 classes.


In [9]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('model.h5')

Epoch 1/50
100/100 [==============================] - 51s 512ms/step - loss: 0.7312 - acc: 0.4931 - val_loss: 0.6893 - val_acc: 0.5663
Epoch 2/50
100/100 [==============================] - 49s 485ms/step - loss: 0.6996 - acc: 0.5381 - val_loss: 0.6797 - val_acc: 0.5781
Epoch 3/50
100/100 [==============================] - 48s 484ms/step - loss: 0.7030 - acc: 0.5850 - val_loss: 0.6409 - val_acc: 0.5885
Epoch 4/50
100/100 [==============================] - 48s 484ms/step - loss: 0.6602 - acc: 0.6044 - val_loss: 0.6276 - val_acc: 0.6250
Epoch 5/50
100/100 [==============================] - 49s 486ms/step - loss: 0.6468 - acc: 0.6363 - val_loss: 0.6527 - val_acc: 0.6641
Epoch 6/50
100/100 [==============================] - 49s 488ms/step - loss: 0.6297 - acc: 0.6838 - val_loss: 0.5827 - val_acc: 0.7344
Epoch 7/50
100/100 [==============================] - 49s 492ms/step - loss: 0.6241 - acc: 0.6787 - val_loss: 0.5646 - val_acc: 0.7318
Epoch 8/50
100/100 [==============================] - 5

In [10]:
# dimensions of our images
img_width, img_height = 150, 150

# load the model we saved
model = load_model('model.h5')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

mypath = "predict/"  # Location of the images to be predicted
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
# predicting images
dog_counter = 0 
cat_counter  = 0
for file in onlyfiles:
    img = image.load_img(mypath+file, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    classes = classes[0][0]
    
    if classes == 0:
        print(file + ": " + 'cat')
        cat_counter += 1
    else:
        print(file + ": " + 'dog')
        dog_counter += 1
print("Total Dogs :",dog_counter)
print("Total Cats :",cat_counter)

['8.jpg', '4.jpg', '5.jpg', '7.jpg', '6.jpg', '2.jpg', '3.jpg', '1.jpg']
8.jpg: dog
4.jpg: cat
5.jpg: dog
7.jpg: cat
6.jpg: cat
2.jpg: dog
3.jpg: cat
1.jpg: dog
Total Dogs : 4
Total Cats : 4
